In [1]:
import numpy as np
import cProfile
from operator import mul

# Fast Exponentiation

We can quickly compute $a^b \ mod\ m$ using, the following trick best illustrated by example.

$$
5^{13} = 5 * 5^{12} = 5 * (5^6)^2 = 5 * ((5^3)^2)^2 = 5 * ((5*5^2)^2)^2 = 5 * ((5*5*5)^2)^2
$$

In that example even though we would naively need 11 multiplications to calculate results we managed to get away with 5.
In general we can write out 

$$
fexp(a,b,m) =
\begin{cases}
a & \text{if}\ b=1\\
fexp(a,b/2,m)^2 &\text{if}\ b\ \text{even}\\
a \cdot fexp(a,b-1,m) &\text{otherwise}
\end{cases}
$$


In [2]:
def fexp_recursive(a, b, m, mul_op=mul):
    # We can easiely handle b = 0, here, but we choose not to
    # this will be helpful later when we deal with matrices...
    assert b >= 1
    if b == 1:
        return a
    elif b % 2 == 0:
        conquered = fexp_recursive(a, b / 2, m, mul_op=mul_op)
        return  mul_op(conquered, conquered) % m
    else:
        b_one_less = fexp_recursive(a, b - 1, m, mul_op=mul_op)        
        return mul_op(a, b_one_less) % m

In [3]:
fexp_recursive(2, 6, 1000)

64

In [4]:
fexp_recursive(2, 6, 10)

4

In [5]:
cProfile.run("fexp_recursive(2, 10000000000, 10)")

         89 function calls (46 primitive calls) in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     44/1    0.000    0.000    0.000    0.000 <ipython-input-2-78a9a64277e1>:1(fexp_recursive)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       43    0.000    0.000    0.000    0.000 {operator.mul}




### Iterative approach
Very similar algorithm can be written out without recursion by looking at binary representation of b and noticing that if $i-th$ bit is one, then we need to multiply the result by $$a^{2^i}$$
Don't worry if you don't fully understand the code below. It is included here, to show you that there are multiple ways of approaching implementation of this kind of solution. Also iterative algorithms are sometimes preferred - we will get back to that point below.

In [6]:
def fexp_iterative(a, b, m, mul_op=mul):
    assert b >= 1
    result = a
    multiplier = a
    b -= 1
    while b > 0:
        if b % 2 == 1:
            result = mul_op(result, multiplier) % m
        multiplier = mul_op(multiplier, multiplier) % m
        b /= 2
    return result

In [7]:
fexp_iterative(2, 6, 1000)

64

In [8]:
fexp_iterative(2, 6, 10)

4

In [9]:
cProfile.run("fexp_iterative(2, 10000000000, 10)")

         57 function calls in 0.000 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.000    0.000    0.000    0.000 <ipython-input-6-dc8e4fdca639>:1(fexp_iterative)
        1    0.000    0.000    0.000    0.000 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       54    0.000    0.000    0.000    0.000 {operator.mul}




# Fibonacci sequence

The Fibonacci Sequence is the series of numbers: 0, 1, 1, 2, 3, 5, 8, 13, 21, 34, ... The next number is found by adding up the two numbers before it. I.e. 3 is found by adding the two numbers before it (1+2). Here we will explore 3 different algorithms for computing the $n^{th}$ Fibonacci number and analyze their time complexity. We denote the $n^{th}$ Fibonacci number as $F_{n}$. Code for the following 3 algorithms is in recitation1.py which is available on the Stellar site under recitation materials. 


### Naive Recursion

By definition, $F_{n} = F_{n - 1} + F_{n - 2}$. As this is the ``naive'' algorithm, let's not try to be too clever and instead simply write an algorithm using only this definition!

Now to analyze the runtime. Formally this algorithm can be analyzed by solving the recurrence, $T(n) = T(n - 1) + T(n - 2) + \Theta(1)$. This is a tough recursion to solve! Let us separately find an upper and lower bound instead of a $\Theta$ relation. 

It is clear that the recurrence $T(n) = 2T(n - 1) + \Theta(1)$ is strictly greater than our original, so let us use it to find an upper bound. Each recursive call results in two child recursive calls until the base case is reached. Therefore, there will be $\Theta(2^{i})$ recursive calls made at the $i^{th}$ level of recursion. Since, the subproblem size only decreases by one on each call, there will be $\Theta(n)$ levels of recursion before the base case is reached. Therefore this recurrence solves to be $\Theta(2^{n})$ and we can conclude that our algorithm is $O(2^{n})$

The recurrence $T(n) = 2T(n - 2) + \Theta(1)$ is strictly less than our original. Using similar logic as above we can see that this recurrence solves to $\Theta(2^{\frac{n}{2}})$ and we conclude that our algorithm is $\Omega(2^{\frac{n}{2}})$.

Challenge Problem: Find a tight asymptotic bound to this algorithms runtime. Hint: Draw a tree diagraming recursive calls and look for the pattern!



In [10]:
def fibonacci_recursive_slow(n, m):
    assert n >= 0
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        return (fibonacci_recursive_slow(n - 1, m) + fibonacci_recursive_slow(n - 2, m)) % m 

In [11]:
fibonacci_recursive_slow(10, 1000)

55

In [12]:
cProfile.run("fibonacci_recursive_slow(30, 1000)")

         2692539 function calls (3 primitive calls) in 0.888 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
2692537/1    0.888    0.000    0.888    0.888 <ipython-input-10-c32dd02825c7>:1(fibonacci_recursive_slow)
        1    0.000    0.000    0.888    0.888 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




### Memoized Recursion
It's often the case that we can improve the efficiency of algorithms by exploiting natural ``structures'' present in the problem. Notice in the naive algorithm that we often compute the same thing multiple times! This occurs because we have overlapping subproblems. For example, both $F_{n - 1}$ and $F_{n - 2}$ depend on the solution to $F_{n - 3}$. We can take advantage of this structure by memoizing (storing) the solutions to subproblems as we go. Therefore instead of recalculating them we can simply look them up! Look in recitation1.py for python code.

This improved algorithm has a time complexity of $\Theta(n)$. This can be seen from the fact that we in total solve for $\Theta(n)$ $F_{i}$s, each of which take only constant time to compute. 


In [13]:
cache = {}
def fibonacci_recursive_fast(n, m):
    if not (n,m) in cache:
        assert n >= 0
        if n == 0:
            result = 0
        elif n == 1:
            result = 1
        else:
            result = (fibonacci_recursive_fast(n - 1, m) + fibonacci_recursive_fast(n - 2, m)) % m
        cache[(n,m)] = result
    return cache[(n,m)]

In [14]:
fibonacci_recursive_fast(10, 1000)

55

In [15]:
cProfile.run("fibonacci_recursive_fast(900, 1000)")

         1783 function calls (3 primitive calls) in 0.004 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
   1781/1    0.004    0.000    0.004    0.004 <ipython-input-13-45ec8644bee2>:2(fibonacci_recursive_fast)
        1    0.000    0.000    0.004    0.004 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




### Iterative versus recursive solutions
The code below should give a runtime error on a standard Python interpreter - because its exceeding the default stack limit. This kind of limitation is why we often opt for iterative versions of the algorithm. Don't worry though, it turns out that for every recursive solution there exists an itertive equivalent. Indeed - we can emulate recursion stack with a stack datastructure. Such a solution is often tedious to implemented and constact factor of the runtime become large. There's why we often seek for *natural order of calculation*, i.e. order in which we compute the subproblems, such that by the time we need a particular result it has alredy been computed. For example in case of Fibonacci the natural order of computatation is to compute $F_1$, then $F_2$, then $F_3$ etc. Notice how resulting solution is even simpler than the recursive one!

In [16]:
# fibonacci_recursive_fast(9000, 1000)

In [17]:
def fibonnaci_iterative(n, m):
    assert n >= 0
    if n == 0:
        return 0
    f_current, f_previous = 1, 0
    for _ in range(n - 1):
        f_current, f_previous = f_current + f_previous % m, f_current
    return f_current

In [18]:
fibonnaci_iterative(10, 1000)

55

In [19]:
cProfile.run("fibonnaci_iterative(10000000, 1000)")

         4 function calls in 1.025 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.901    0.901    1.025    1.025 <ipython-input-17-732ce64d7038>:1(fibonnaci_iterative)
        1    0.000    0.000    1.025    1.025 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
        1    0.124    0.124    0.124    0.124 {range}




### Matrix exponentiation

Take a moment to think back to the recursive squaring algorithm from lecture. In a similar fashion, we can compute the $n^{th}$ Fibonacci number in logarithmic time by repeatedly squaring the matrix 


\begin{align}
\begin{bmatrix}
    1 & 1 \\
    1 & 0
\end{bmatrix}
\end{align}

In fact


\begin{align}
\begin{bmatrix}
    1 & 1 \\
    1 & 0
\end{bmatrix} ^{n} 
= 
\begin{bmatrix}
    F_{n + 1} & F_{n} \\
    F_{n} & F_{n - 1}
\end{bmatrix}
\end{align}

To give a rough proof of why this is the case, let us use induction on $n$. Our claim is trivially true in the base case $n = 1$. Now assuming that our claim holds for this matrix to the $n^{th}$ power, we must show that our claim is also true for this matrix to the $(n + 1)^{th}$ power. 


\begin{align}
\begin{bmatrix}
    1 & 1 \\
    1 & 0
\end{bmatrix}
*
 \begin{bmatrix}
    1 & 1 \\
    1 & 0 
\end{bmatrix} ^{n} 
= 
\begin{bmatrix}
    1 & 1 \\
    1 & 0 \\
\end{bmatrix}
*
\begin{bmatrix}
    F_{n + 1} & F_{n} \\
    F_{n} & F_{n - 1}
\end{bmatrix}
 = 
 \begin{bmatrix}
    F_{n + 1} + F_{n} & F_{n} + F_{n - 1} \\
    F_{n + 1} & F_{n}
\end{bmatrix}
=
\begin{bmatrix}
    F_{n + 2} & F_{n+1} \\
    F_{n + 1} & F_{n }
\end{bmatrix}
\end{align}

Success! 

The runtime analysis for this algorithm is identical to that for modular exponentiation using repeated squaring. In particular we do not include cost of matrix multiply in our analysis, because matrix has constant size.

In [23]:
F = np.array([[1, 1],
              [1, 0]])

def fibonnaci_matrix(n, m):
    Fn = fexp_recursive(F, n, m, mul_op=np.dot)
    return Fn[0][1]

In [24]:
fibonnaci_matrix(10, 1000)

55

In [25]:
cProfile.run("fibonnaci_matrix(10000000000, 1000)")

         90 function calls (47 primitive calls) in 0.001 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     44/1    0.000    0.000    0.001    0.001 <ipython-input-2-78a9a64277e1>:1(fexp_recursive)
        1    0.000    0.000    0.001    0.001 <ipython-input-23-eff9346cd7e3>:4(fibonnaci_matrix)
        1    0.000    0.000    0.001    0.001 <string>:1(<module>)
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}
       43    0.000    0.000    0.000    0.000 {numpy.core.multiarray.dot}




# Problems to think about (non-examinable, non-compulsory, strictly for fun...)

1. Give an example of another operation besides multiplication and matrix multiply that can be efficiently composed using fast exponentiation.

2. Compute the n-th item of tribonacci sequence using the three methods presented above:

\begin{align}
s_n =
\begin{cases}
1 & \text{if}\ n \in \{ 1,2,3 \} \\
2s_{n-1} + 2s_{n-2} + s_{n-3} & \text{otherwise}
\end{cases}
\end{align}
